<a href="https://colab.research.google.com/github/patty1112/disasterTweets/blob/main/470Kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Run to download all files. Files will be located in a folder called "kaggle npl"
!mkdir ~/.kaggle
!echo '{"username":"michaelbasey","key":"726dfabbae8737ae9006a4fd57b07bfb"}' > ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json
!pip install kaggle
!kaggle datasets download -d dipankarmitra/natural-language-processing-with-disaster-tweets
!unzip natural-language-processing-with-disaster-tweets.zip

  0% 0.00/607k [00:00<?, ?B/s]
100% 607k/607k [00:00<00:00, 108MB/s]
Archive:  natural-language-processing-with-disaster-tweets.zip
  inflating: kaggle nlp/Natural Language Processing with Disaster Tweets (1).ipynb  
  inflating: kaggle nlp/sample_submission.csv  
  inflating: kaggle nlp/test.csv     
  inflating: kaggle nlp/train.csv    


In [2]:
# import necessary libraries
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import sklearn as sk
import re
import nltk
import math
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize
from collections import defaultdict, Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, precision_score, recall_score



nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [3]:
# reads in the training data
df = pd.read_csv('kaggle nlp/train.csv')
df1 = pd.read_csv('kaggle nlp/test.csv')

FileNotFoundError: ignored

In [ ]:
# now, lets pre-process the data

# remove anything that is non-alphanumeric
def remove_non_alpha(text):
    return re.sub(r'[^a-zA-Z\s]', '', text)
df['text'] = df['text'].apply(remove_non_alpha)
df1['text'] = df['text'].apply(remove_non_alpha)

# remove casing
def remove_casing(text):
    return text.lower()
df['text'] = df['text'].apply(remove_casing)
df1['text'] = df['text'].apply(remove_casing)
# tokenize the text

df['tokenized_text'] = df['text'].apply(word_tokenize)
df1['tokenized_text'] = df['text'].apply(word_tokenize)
# remove filler words such as "the, is"

stop_words = set(stopwords.words('english'))

def filter_tokens(tokenized_text):
    return [token for token in tokenized_text if token.lower() not in stop_words]

df['filtered_text'] = df['tokenized_text'].apply(filter_tokens)
df1['filtered_text'] = df['tokenized_text'].apply(filter_tokens)
# remove tenses from words "was" and "is" get treated the same
# helps us to simplify our model.

stemmer = PorterStemmer()

def stem_tokens(tokens):
    return [stemmer.stem(token) for token in tokens]
df['stemmed_text'] = df['filtered_text'].apply(stem_tokens)
df1['stemmed_text'] = df['filtered_text'].apply(stem_tokens)

df.head(1)

,id,keyword,location,text,target,tokenized_text,filtered_text,stemmed_text
0,1,NaN,NaN,our deeds are the reason of this earthquake ma...,1,"[our, deeds, are, the, reason, of, this, earth...","[deeds, reason, earthquake, may, allah, forgiv...","[deed, reason, earthquak, may, allah, forgiv, us]"


In [ ]:
# still haven't decided if I want to lemmatize the text
# im not sure if it helps us tremendously
lemmatizer = WordNetLemmatizer()
def lemmatize_tokens(tokens):
    return [lemmatizer.lemmatize(token) for token in tokens]
df['lemmatized_text'] = df['filtered_text'].apply(lemmatize_tokens)
df1['lemmatized_text'] = df['filtered_text'].apply(lemmatize_tokens)

In [ ]:
# then, we need to create a dictionary of how often each word appears
# in each type of tweet

def generate_frequency(df):
    # Initialize dictionary to store word frequencies for both categories
    freq_dict_combined = defaultdict(lambda: [0, 0])

    for index, row in df.iterrows():
        target = row['target']
        tokens = row['stemmed_text']

        for token in tokens:
            if target == 1:
                freq_dict_combined[token][0] += 1  # Increment disaster count
            elif target == 0:
                freq_dict_combined[token][1] += 1  # Increment non-disaster count

    return freq_dict_combined

freq_dict = generate_frequency(df)

# Convert defaultdict to regular dictionary for better readability
freq_dict = {k: tuple(v) for k, v in dict(freq_dict).items()}

# Print the results
print("Frequency Dictionary:")
print(freq_dict)

Frequency Dictionary:
{'deed': (1, 1), 'reason': (8, 23), 'earthquak': (47, 6), 'may': (50, 38), 'allah': (6, 3), 'forgiv': (1, 2), 'us': (84, 86), 'forest': (50, 16), 'fire': (269, 90), 'near': (49, 7), 'la': (25, 5), 'rong': (1, 0), 'sask': (1, 0), 'canada': (10, 3), 'resid': (9, 0), 'ask': (5, 23), 'shelter': (6, 0), 'place': (17, 18), 'notifi': (1, 0), 'offic': (41, 11), 'evacu': (101, 29), 'order': (24, 14), 'expect': (22, 11), 'peopl': (106, 93), 'receiv': (4, 0), 'wildfir': (80, 3), 'california': (115, 6), 'got': (32, 91), 'sent': (4, 9), 'photo': (28, 37), 'rubi': (1, 1), 'alaska': (6, 2), 'smoke': (14, 40), 'pour': (2, 2), 'school': (32, 38), 'rockyfir': (4, 0), 'updat': (36, 19), 'hwi': (10, 0), 'close': (29, 9), 'direct': (6, 12), 'due': (24, 7), 'lake': (9, 7), 'counti': (31, 7), 'cafir': (2, 0), 'flood': (120, 47), 'disast': (118, 39), 'heavi': (18, 2), 'rain': (38, 20), 'caus': (63, 41), 'flash': (17, 4), 'street': (24, 8), 'manit': (1, 0), 'colorado': (10, 6), 'spring': 

In [ ]:
# Now, lets find the probability of the occurence given word_i

def generate_relative_freq(df):
    # Initialize dictionary to store word relative frequency for both categories
    rel_freq = defaultdict(lambda: [0.0, 0.0])

    disaster_words = 0
    non_disaster_words = 0
    vocab_size = 0  # Size of the vocabulary (unique words in all categories)

    for index, row in df.iterrows():
        label = row['target']
        tokens = row['stemmed_text']

        # Count total words in each category
        if label == 1:
            disaster_words += len(tokens)
        elif label == 0:
            non_disaster_words += len(tokens)

        # Count occurrences of each word in each category
        for token in tokens:
            if label == 1:
                rel_freq[token][0] += 1  # Increment disaster count
            elif label == 0:
                rel_freq[token][1] += 1  # Increment non-disaster count

    # Count the size of the vocabulary (unique words)
    vocab_size = len(rel_freq.keys())

    # Convert counts to relative frequencies with Laplace smoothing
    for word, counts in rel_freq.items():
        rel_freq[word][0] = (counts[0] + 1) / (disaster_words + vocab_size)
        rel_freq[word][1] = (counts[1] + 1) / (non_disaster_words + vocab_size)

    return rel_freq

rel_dict = generate_relative_freq(df)
# Convert defaultdict to regular dictionary for better readability
rel_dict = {k: tuple(v) for k, v in dict(rel_dict).items()}

# Print the results
print("Relative Frequency Dictionary:")
print(rel_dict)

Relative Frequency Dictionary:
{'deed': (3.84201629014907e-05, 3.404023555843007e-05), 'reason': (0.00017289073305670817, 0.00040848282670116076), 'earthquak': (0.0009220839096357768, 0.00011914082445450522), 'may': (0.000979714153988013, 0.0006637845933893863), 'allah': (0.00013447057015521745, 6.808047111686014e-05), 'forgiv': (3.84201629014907e-05, 5.1060353337645095e-05), 'us': (0.0016328569233133548, 0.0014807502467917078), 'forest': (0.000979714153988013, 0.00028934200224665555), 'fire': (0.005186721991701245, 0.0015488307179085678), 'near': (0.0009605040725372676, 0.00013616094223372027), 'la': (0.0004994621177193792, 0.00010212070667529019), 'rong': (3.84201629014907e-05, 1.7020117779215034e-05), 'sask': (3.84201629014907e-05, 1.7020117779215034e-05), 'canada': (0.00021131089595819886, 6.808047111686014e-05), 'resid': (0.00019210081450745352, 1.7020117779215034e-05), 'ask': (0.0001152604887044721, 0.00040848282670116076), 'shelter': (0.00013447057015521745, 1.7020117779215034e-

In [ ]:
def naive_bayes_classify(tokenized_tweet, rel_freq_dict, p_disaster=0.5, p_non_disaster=0.5):
    # Initialize log probabilities to 0
    log_prob_disaster = math.log(p_disaster)
    log_prob_non_disaster = math.log(p_non_disaster)

    # Laplace-smoothed probabilities for unseen words
    unseen_prob_disaster = 1 / (sum(rel_freq_dict[word][0] for word in rel_freq_dict) + len(rel_freq_dict))
    unseen_prob_non_disaster = 1 / (sum(rel_freq_dict[word][1] for word in rel_freq_dict) + len(rel_freq_dict))

    for token in tokenized_tweet:
        # Fetch probabilities from dictionary, falling back to unseen probabilities
        prob_disaster, prob_non_disaster = rel_freq_dict.get(token, (unseen_prob_disaster, unseen_prob_non_disaster))

        # Add log probabilities
        log_prob_disaster += math.log(prob_disaster)
        log_prob_non_disaster += math.log(prob_non_disaster)

    # Choose label with highest log probability
    if log_prob_disaster > log_prob_non_disaster:
        return 1  # Disaster
    else:
        return 0  # Non-disaster


In [ ]:
def evaluate_model(df, rel_freq_dict):
    true_labels = []
    predicted_labels = []

    # Loop through each row in the DataFrame to get tokenized text and target label
    for index, row in df.iterrows():
        tokenized_tweet = row['stemmed_text']
        true_label = row['target']

        # Classify the tweet
        predicted_label = naive_bayes_classify(tokenized_tweet, rel_freq_dict)

        # Store true and predicted labels
        true_labels.append(true_label)
        predicted_labels.append(predicted_label)

    # Calculate F1 score, precision, and recall
    f1 = f1_score(true_labels, predicted_labels)
    precision = precision_score(true_labels, predicted_labels)
    recall = recall_score(true_labels, predicted_labels)

    return f1, precision, recall

# Your DataFrame should be in the variable df and your relative frequency dictionary in rel_freq_dict



In [ ]:
    f1, precision, recall = evaluate_model(df, rel_dict)
    print(f"F1 Score: {f1}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")

F1 Score: 0.8942398489140699
Precision: 0.9215050275705482
Recall: 0.8685417303576888


In [ ]:
# now, lets see how we do on the test data. it is not categorized.
predicted_labels = []
for index, row in df1.iterrows():
      whole_tweet = row['text'];
      tokenized_tweet = row['stemmed_text']
      predicted_label = naive_bayes_classify(tokenized_tweet, rel_dict)
      predicted_labels.append([whole_tweet, predicted_label])

In [ ]:
print("1 is disaster, 0 is not")
print(predicted_labels)

1 is disaster, 0 is not
[['our deeds are the reason of this earthquake may allah forgive us all', 1], ['forest fire near la ronge sask canada', 1], ['all residents asked to shelter in place are being notified by officers no other evacuation or shelter in place orders are expected', 1], [' people receive wildfires evacuation orders in california ', 1], ['just got sent this photo from ruby alaska as smoke from wildfires pours into a school ', 1], ['rockyfire update  california hwy  closed in both directions due to lake county fire  cafire wildfires', 1], ['flood disaster heavy rain causes flash flooding of streets in manitou colorado springs areas', 1], ['im on top of the hill and i can see a fire in the woods', 0], ['theres an emergency evacuation happening now in the building across the street', 1], ['im afraid that the tornado is coming to our area', 1], ['three people died from the heat wave so far', 1], ['haha south tampa is getting flooded hah wait a second i live in south tampa wh